## Przygotowanie danych

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import pickle
from dask_ml import preprocddessing
import datetime

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
cluster = LocalCluster()
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:40489 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.19 GB


In [1]:
import pandas as pd

In [2]:
df_headers = ['Sale','SalesAmountInEuro','time_delay_for_conversion','click_timestamp',
              'nb_clicks_1week','product_price','product_age_group','device_type',
              'audience_id','product_gender','product_brand',
              'prod_category1','prod_category2','prod_category3','prod_category4',
              'prod_category5','prod_category6','prod_category7','product_country',
              'product_id','product_title','partner_id','user_id']

# dtypes={'Sale':'bool','prod_category1': 'object','prod_category2': 'object',
#         'prod_category3': 'object','prod_category4': 'object',
#         'prod_category5': 'object','prod_category6': 'object',
#         'prod_category7': 'object', 'product_category': 'str'}

dtypes={'prod_category1': 'category','prod_category2': 'category',
        'prod_category3': 'category','prod_category4': 'category',
        'prod_category5': 'category','prod_category6': 'category',
        'prod_category7': 'category', 'product_category': 'str'}

encoder_labels = ['product_age_group','device_type','audience_id',
                  'product_gender', 'product_brand','prod_category1','prod_category2',
                  'prod_category3','prod_category4','prod_category5',
                  'prod_category6','prod_category7','product_country',
                  'product_id','product_title','partner_id','user_id']

def dateparse (time_in_secs):
    return datetime.datetime.fromtimestamp(float(time_in_secs))

df = pd.read_csv("../../../data/CriteoSearchData.csv", header=None,
                 names=df_headers, delimiter='\t', dtype=dtypes)

In [3]:
df = df[df['partner_id']=='017EBBBE0B84EC3065F8A4FAFA39A4B2']

In [5]:
df

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,prod_category3,prod_category4,prod_category5,prod_category6,prod_category7,product_country,product_id,product_title,partner_id,user_id
10444,0,-1.0,-1,1598842113,-1,0.0,-1,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,0135912E122DA5C7531FC04BCEB988D0
11285,0,-1.0,-1,1598824176,-1,0.0,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,BBBAD5A6C857F2988E8E9D66C0361A19
87581,0,-1.0,-1,1600448810,-1,0.0,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,0B402E9643D1C91FF8EAC5A0224392B3
136195,0,-1.0,-1,1598665248,-1,0.0,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,8621838CFEBC44C6D95D6A2213EC8788
173952,0,-1.0,-1,1601044094,-1,0.0,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,D6C27E386E8600D508EFB88AF79785A3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15811875,0,-1.0,-1,1597757649,-1,0.0,-1,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,FD3B5557ED19D85E7A5F361FFCEA2A30
15815742,0,-1.0,-1,1596456994,-1,0.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,82B4439394A06EFCD16C5FC9E8CA74F3
15838663,0,-1.0,-1,1597116382,-1,0.0,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,235FD301E045B659038CF8C46897B7B7
15908882,0,-1.0,-1,1601856966,-1,0.0,-1,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,017EBBBE0B84EC3065F8A4FAFA39A4B2,B02E9CDF6A0D1F93DD355CC0CEB64EC4


In [6]:
df.to_csv('017EBBBE0B84EC3065F8A4FAFA39A4B2raw.csv')

In [4]:
df['product_title'] = df.product_title.astype(str)
df['product_title'] = df['product_title'].replace('nan', 'BROKEN')

In [5]:
df['product_title'] = df.product_title.astype(str)
df['user_id'] = df.user_id.astype(str)


### Praca z dask_ml.preprocessing.LabelEncoder

#### Przygotowanie LabelEncodera

In [ ]:
# not needed anymore, load pickle file instead
labelEncoders = {x:preprocessing.LabelEncoder() for x in encoder_labels}
for k,v in labelEncoders.items():
    print('starting {}'.format(k))
    df[k] = v.fit_transform(df[k])
    print('done')

#### Zapis labelEncodera do pliku

In [31]:
import pickle
with open('../lablencoder_new.pickle', 'wb') as handle: #wiem, że tu jest literówka, trudno
    pickle.dump(labelEncoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Wykorzystanie labelEncodera

Załaduj przygotowany labelEncoder za pomocą:

In [1]:
import pickle
labelEncoders = pickle.load(open("../../../data/lablencoder.pickle","rb"))

In [4]:
labelEncoders

{'product_age_group': LabelEncoder(),
 'device_type': LabelEncoder(),
 'audience_id': LabelEncoder(),
 'product_gender': LabelEncoder(),
 'product_brand': LabelEncoder(),
 'prod_category1': LabelEncoder(),
 'prod_category2': LabelEncoder(),
 'prod_category3': LabelEncoder(),
 'prod_category4': LabelEncoder(),
 'prod_category5': LabelEncoder(),
 'prod_category6': LabelEncoder(),
 'prod_category7': LabelEncoder(),
 'product_country': LabelEncoder(),
 'product_id': LabelEncoder(),
 'product_title': LabelEncoder(),
 'partner_id': LabelEncoder(),
 'user_id': LabelEncoder()}

In [3]:
productidclasslist = list(labelEncoders.get('product_id').classes_)

In [ ]:
productidDict = {productidclasslist.index(x): x for x in productidclasslist}

In [21]:
import pickle
with open('retarded_product_id_dict.pickle', 'wb') as handle: #wiem, że tu jest literówka, trudno
    pickle.dump(partneridDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
productidDict

#### Badanie zawartości labelEncodera

In [11]:
labelEncoders

{'product_age_group': LabelEncoder(),
 'device_type': LabelEncoder(),
 'audience_id': LabelEncoder(),
 'product_gender': LabelEncoder(),
 'product_brand': LabelEncoder(),
 'prod_category1': LabelEncoder(),
 'prod_category2': LabelEncoder(),
 'prod_category3': LabelEncoder(),
 'prod_category4': LabelEncoder(),
 'prod_category5': LabelEncoder(),
 'prod_category6': LabelEncoder(),
 'prod_category7': LabelEncoder(),
 'product_country': LabelEncoder(),
 'product_id': LabelEncoder(),
 'product_title': LabelEncoder(),
 'partner_id': LabelEncoder(),
 'user_id': LabelEncoder()}

In [7]:
for k,v in labelEncoders.items():
    df[k] = v.transform(df[k].values)
    print('{} done'.format(k))

product_age_group done
device_type done
audience_id done
product_gender done
product_brand done
prod_category1 done
prod_category2 done
prod_category3 done
prod_category4 done
prod_category5 done
prod_category6 done
prod_category7 done
product_country done
product_id done
product_title done
partner_id done
user_id done


In [ ]:
for k,v in labelEncoders.items():
    print("{} table lookup".format(k))
    print("showing {} classes:".format(len(v.classes_)))
    print(v.classes_)
    print("\n")

Kolumna product_title zawiera wartość BROKEN zastępującą wcześniej pojawiający sie NaN

In [37]:
'BROKEN' in labelEncoders.get('product_title').classes_

True

In [38]:
labelEncoders.get('product_title').transform(['BROKEN'])

array([596334])

In [39]:
labelEncoders.get('product_title').inverse_transform([596334])

array(['BROKEN'], dtype=object)

tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tornado/websocket.py", line 954, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/tornado/web.py", line 3173, in wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/bokeh/server/views/ws.py", line 137, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [10]:
df.to_parquet('CPStimestampnotformatted.parquet')

In [11]:
df

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,prod_category3,prod_category4,prod_category5,prod_category6,prod_category7,product_country,product_id,product_title,partner_id,user_id
0,0,-1.0,-1,1598891820,-1,0.0,0,3,0,0,...,0,0,0,0,0,8,1058198,0,277,3924600
1,0,-1.0,-1,1598925284,0,0.0,4,7,0,2,...,0,0,0,0,0,8,486749,188114,230,11188022
2,0,-1.0,-1,1598892568,-1,0.0,0,3,0,0,...,0,0,0,0,0,0,0,0,245,12458754
3,0,-1.0,-1,1598891174,-1,0.0,0,7,12657,0,...,0,0,0,0,0,0,0,0,85,8086037
4,0,-1.0,-1,1598905245,-1,0.0,0,3,0,0,...,0,0,0,0,0,8,606352,0,277,6010702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995629,0,-1.0,-1,1596998117,14,0.0,6,3,0,2,...,0,0,0,0,0,22,820656,108102,252,7800357
15995630,0,-1.0,-1,1597018885,0,0.0,4,3,0,2,...,304,1302,0,0,0,22,1414085,100905,88,12516134
15995631,0,-1.0,-1,1596960466,5,0.0,4,3,0,14,...,0,0,0,0,0,22,721542,518847,252,1590464
15995632,0,-1.0,-1,1596960481,5,0.0,4,3,0,14,...,0,0,0,0,0,22,721542,518847,252,1590464
